similarity_to(spec2, round_precision=0)[source]

    Compares two spectra and returns cosine

Parameters

    spec2 (Spectrum) – another pymzml spectrum that is compared to the current spectrum.

Keyword Arguments
    
    round_precision (int) – precision mzs are rounded to, i.e. round( mz, round_precision )

Returns
    
    value between 0 and 1, i.e. the cosine between the two spectra.

Return type
    
    cosine (float)

highest_peaks(n)[source]

Function to retrieve the n-highest centroided peaks of the spectrum.

In [1]:
import pymzml
import matplotlib.pyplot as plt
import numpy as np

import pyteomics
from pyteomics import mzml, auxiliary

import plotly.graph_objects as go

import pandas as pd

In [2]:
path = 'D:/UW/massmotif/mzml/DRO_DIE_1ppm_29h_exp1_method_1.mzML'

In [3]:
f=mzml.MzML(path)

In [4]:
def get_scans(path, ms_all = False, ms_lv = 1):
    run = pymzml.run.Reader(path)
    scans = []
    if ms_all == False:
        for scan in run:
            if scan.ms_level == ms_lv:
                scans.append(scan)
    elif ms_all == True:
        for scan in run:
            scans.append(scan)
            
    return scans

In [5]:
ms2s = get_scans(path, False, 2)

- use first5 rather than precursor alone -> 100 baseline for first5 -> loop until precursor - 100 
- group result with precursor+retention time(0.5 min)

In [8]:
def motif_seek(ms2_scans, motifs, error = 0.002, noise_level = 10, precursor_base = 500, top_frags = 5, precursor_dist = 50, mzrange = [0, 500], rtrange = [0, 20]):
    motif_result = []
    motif_range = []
    for motif in motifs:
        motif_range.append([motif - error, motif + error])
    
    for scan in ms2_scans:
        precursor = scan.selected_precursors[0]['mz']
        drop_index = np.argwhere(scan.i <= noise_level)
        scan.i = np.delete(scan.i, drop_index)
        scan.mz = np.delete(scan.mz, drop_index)
        
        frag = scan.mz[scan.mz < precursor]
        frag_i = scan.i[: len(frag)] # In case need it
        base_index = np.argwhere(frag_i >= precursor_base)
        top_list = sorted(frag[base_index])[-top_frags : ]
        top_range = precursor - precursor_dist
        top_list = [top for top in top_list if top >= top_range]
        
        
        neutral_loss = precursor - frag
        for top_frag in top_list:
            neutral_loss_top = top_frag - frag
            neutral_loss_top = neutral_loss_top[neutral_loss_top > 0]
            neutral_loss = np.append(neutral_loss, neutral_loss_top)
        
        mtf_count = 0
        for mtf in motif_range:
            mtf_hit = neutral_loss[(mtf[0] < neutral_loss) & (neutral_loss < mtf[1])]
            if len(mtf_hit) > 0:
                mtf_count += 1
        
        if mtf_count == len(motifs):
            motif_result.append([scan.selected_precursors[0]['mz'], round(scan.scan_time[0],2), scan.ID])
    
    result = sorted(motif_result)
    d_motif = pd.DataFrame(result, columns = ['mz', 'rt', 'scanID']) 
    d_motif = d_motif[(d_motif['mz'] < mzrange[1]) 
                    & (d_motif['mz'] > mzrange[0])
                    & (d_motif['rt'] > rtrange[0])
                    & (d_motif['rt'] < rtrange[1])]
    d_motif.sort_values(by=['mz', 'rt'], inplace=True)
    d_motif.reset_index(inplace=True)
    d_motif.drop(columns = ['index'], inplace=True)

    return d_motif

In [9]:
motif = [41.0265, 18.0105]
#motif = [41.0265]
result = motif_seek(ms2s, motif, error = 0.01, noise_level = 10, precursor_base = 500, mzrange = [200, 400], rtrange = [2, 10])

In [39]:
def motif_export(motif_result, export_name='Motif.xlsx'):
    writer = pd.ExcelWriter(export_name, engine='xlsxwriter')
    for i,j in motif_result.groupby('mz'):
        d_iter = pd.DataFrame(j)
        d_iter.to_excel(writer, sheet_name=str(round(i,2)))
    writer.save()
    
    return

In [40]:
motif_export(result)

In [ ]:
def find_scan(ms2s, scanid, interactive = True):
    
    for scan in ms2s:
        if scan.ID == scanid:
            break
    
    print('Precursor m/z: {:0.2f}, Scan time: {:0.1f} minute'.format(scan.selected_precursors[0]['mz'], scan.scan_time[0]))
    
    mz = scan.mz
    ints = scan.i
    
    if interactive == True:
        plt.clf()
        fig = go.Figure([go.Bar(x=mz, y=ints, marker_color = 'red', width = 0.5,
                        hovertemplate =
                        'Int: %{y}'+
                        '<br>m/z: %{x}<br>')])
        fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(0,0,0)',
                  marker_line_width=0.5, opacity=1)
        fig.update_layout(
                template = 'simple_white',
                width = 1000,
                height = 600,
                xaxis = dict(title = 'm/z ratio',
                        rangeslider=dict(
            visible = True
        )),
                yaxis = dict(
                    title = 'Intensity'))
        fig.show()
    
    elif interactive == False:
        plt.figure(figsize=(10,5))
        plt.bar(mz, ints, width = 1.0)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        plt.xlabel('m/z')
        plt.ylabel('Intensity')
        plt.title('MS1 spectrum')
        plt.xlim(0,340)

    return 

In [ ]:
test1 = find_scan(ms2s, 354961, True)

In [ ]:
def cos_one(input_scan, ms2_scans, score_thres = 0.9):
    count = 0
    for scan in ms2_scans:
        score = input_scan.similarity_to(scan, round_precision=0)
        if score >= score_thres:
            count +=1
            print(count)
    print('one scan finished')
                        
    return count

In [ ]:
f[60]